<a href="https://colab.research.google.com/github/qurat-shakeel/LangChain_/blob/main/RAG_project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
! pip install langchain

In [21]:
! pip install -qU langchain-openai

In [22]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()


··········


In [23]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

··········


In [24]:
!pip install pdfplumber

In [25]:
!pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-chroma

In [26]:
from langchain.schema import Document
import pdfplumber
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [27]:
# Custom loader for PDF files
class PDFLoader:
    def __init__(self, file_path):
        self.file_path = file_path

    def load(self):
        try:
            full_text = []
            with pdfplumber.open(self.file_path) as pdf:
                for page in pdf.pages:
                    full_text.append(page.extract_text())
            return "\n".join(full_text)
        except Exception as e:
            raise RuntimeError(f"Error loading {self.file_path}") from e


In [28]:
file_path = "whhispering grove.pdf"

# Load the PDF file
text_content = PDFLoader(file_path=file_path).load()


# Wrap text content in Document objects
documents = [Document(page_content=text_content)]

# Split the text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(documents)

In [29]:
documents = loader.load()
print(documents)

NameError: name 'loader' is not defined

In [30]:
print(documents)

[Document(page_content='Title: The Whispering Grove\nIn a small village nestled between two towering mountains, there was a mysterious grove known\nas the Whispering Grove. The villagers spoke of it in hushed tones, warning their children never\nto venture too close. They said the trees could speak, though no one could ever quite understand\nwhat they were saying.\nOne day, a young girl named Elara, curious and brave, decided to explore the grove. She had\nalways been drawn to the stories of the whispering trees, and her curiosity was too strong to\nresist. Armed with nothing but her wits and a small lantern, she ventured into the grove at dusk.\nAs she walked deeper into the forest, the air grew colder, and the light from her lantern seemed\nto dim. The trees around her began to murmur, their leaves rustling with strange, unintelligible\nwhispers. Elara felt a shiver run down her spine, but she pressed on, determined to uncover the\nsecrets of the grove.\nAfter what felt like hours, s

In [31]:
print(len(splits))

3


In [32]:
for i, split in enumerate(splits):
    print(f"split {i + 1}:")
    print(splits)
    print("-" * 30)

split 1:
[Document(page_content='Title: The Whispering Grove\nIn a small village nestled between two towering mountains, there was a mysterious grove known\nas the Whispering Grove. The villagers spoke of it in hushed tones, warning their children never\nto venture too close. They said the trees could speak, though no one could ever quite understand\nwhat they were saying.\nOne day, a young girl named Elara, curious and brave, decided to explore the grove. She had\nalways been drawn to the stories of the whispering trees, and her curiosity was too strong to\nresist. Armed with nothing but her wits and a small lantern, she ventured into the grove at dusk.\nAs she walked deeper into the forest, the air grew colder, and the light from her lantern seemed\nto dim. The trees around her began to murmur, their leaves rustling with strange, unintelligible\nwhispers. Elara felt a shiver run down her spine, but she pressed on, determined to uncover the\nsecrets of the grove.'), Document(page_cont

In [33]:
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

In [34]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use five sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [35]:
# Create question-answering chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [36]:
response = rag_chain.invoke({"input": "Who is Ellara?"})
response["answer"]

"Elara is a young girl from a small village who is curious and brave. She decides to explore the Whispering Grove, drawn by the stories of the whispering trees. During her adventure, she encounters an ancient tree that communicates with her, revealing the grove's dark history involving a powerful sorcerer and a curse. After learning that the curse cannot be broken, Elara promises never to return to the grove. Her experience leaves her quieter and more introspective, carrying the secret of the grove with her."

In [37]:
response = rag_chain.invoke({"input": "Who is Sorcerer?"})
response["answer"]

'The sorcerer mentioned in the context is a powerful figure associated with the Whispering Grove who conducted experiments there. He imbued the trees with the ability to speak, but something went wrong during his experiments, leading to his disappearance and leaving the grove cursed. The ancient tree in the grove tells Elara about this sorcerer and the history of the grove.'

In [38]:
response = rag_chain.invoke({"input": "Did Ellara returned from Forest?"})
response["answer"]

"Yes, Elara returned from the Whispering Grove after agreeing to the tree's warning. She left the grove unharmed but promised never to return. However, she carried the secret of what she had experienced with her."

In [40]:
response = rag_chain.invoke({"input": "Why it still remained a mystery??"})
response["answer"]

"The Whispering Grove remained a mystery because Elara, after her encounter with the ancient tree and the curse, chose never to speak of what she had experienced. Her silence and introspective nature led the villagers to sense that she was carrying a secret. Additionally, the stories of the grove and its whispering trees were already shrouded in intrigue, and Elara's experience only deepened that mystery. The curse prevented anyone from breaking the silence surrounding the grove's true nature. Thus, the secrets of the grove remained known only to the ancient trees and Elara."

In [41]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [42]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [43]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

question = "Who is Elara?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=ai_msg_1["answer"]),
    ]
)

second_question = "Was she Brave?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

Yes, Elara was brave. She decided to explore the mysterious Whispering Grove despite warnings from the villagers. Her curiosity drove her to venture into a place that was considered dangerous, demonstrating her willingness to confront the unknown. Even as she felt fear while hearing the whispers of the trees, she pressed on to uncover the grove's secrets.


In [44]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [45]:
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [46]:
conversational_rag_chain.invoke(
    {"input": "Why were villagers afraid?"},
    config={
        "configurable": {"session_id": "A1"}
    },
)["answer"]

"Villagers were afraid of the Whispering Grove because they believed it was cursed and warned their children never to venture too close. They spoke of the trees in hushed tones, mentioning that the trees could speak, although no one could understand what they were saying. The mysterious nature of the grove and its whispers created an atmosphere of fear and intrigue. The stories of a powerful sorcerer and the curse added to the sense of danger surrounding the grove. Ultimately, the fear stemmed from the unknown and the potential consequences of disturbing the grove's secrets."

In [48]:
conversational_rag_chain.invoke(
    {"input": "Does Sorcerer  use the grove?"},
    config={"configurable": {"session_id": "A1"}},
)["answer"]

"Yes, the sorcerer used the grove to conduct experiments, which included imbuing the trees with the ability to speak. However, something went wrong during these experiments, leading to the sorcerer's disappearance and the grove being left cursed."

In [49]:
conversational_rag_chain.invoke(
    {"input": "Who is Elara?"},
    config={
        "configurable": {"session_id": "A3"}
    },
)["answer"]

'Elara is a young girl from a small village who is curious and brave. She decides to explore the Whispering Grove, which is known for its mysterious, speaking trees. Driven by her curiosity about the stories surrounding the grove, she ventures into it despite warnings from the villagers. During her exploration, she encounters an ancient tree that communicates with her, revealing a story about a powerful sorcerer and a curse on the grove. After learning about the curse, Elara promises never to return to the grove.'

In [50]:
conversational_rag_chain.invoke(
    {"input": "What happened when she returned from the forest?"},
    config={
        "configurable": {"session_id": "A3"}
    },
)["answer"]

"When Elara returned from the forest, she quickly made her way back to the village after agreeing to the tree's warning to never return to the grove. The villagers noticed that she had grown quieter and more introspective, suggesting she was carrying a secret from her experience. Although she never spoke of what she had heard in the grove, her demeanor changed, and it seemed as if she held a burden that she would take to her grave. The Whispering Grove remained a mystery, with its secrets known only to the ancient trees and Elara."